In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14028445905683370133
xla_global_id: -1
]


2022-06-09 20:36:57.232880: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#{"R": [R], "D": [D],"H": [H], "F":[log10(f)],"RP": [RP]})
DATA = pd.read_csv("DATA/data.csv")
DATA.pop("Unnamed: 0")

ROW_NUM = len(DATA.index)
print(ROW_NUM)
TRAINING_NUM = int(ROW_NUM / 10 * 7)
VALIDATION_NUM = int(ROW_NUM / 20 * 3 )

TRAINING_DATA = DATA.iloc[ :TRAINING_NUM,:]
VALIDATION_DATA = DATA.iloc[TRAINING_NUM:TRAINING_NUM+VALIDATION_NUM,:]
TEST_DATA = DATA.iloc[TRAINING_NUM+VALIDATION_NUM:,:]
TRAINING_VAL = pd.DataFrame(TRAINING_DATA.pop('RP'))
VALIDATION_VAL = pd.DataFrame(VALIDATION_DATA.pop('RP'))
TEST_VAL = pd.DataFrame(TEST_DATA.pop('RP'))

DATA.pop('RP')


282240


0        -123.180388
1        -119.671868
2        -130.180711
3        -111.274042
4        -105.454858
             ...    
282235   -120.310835
282236   -114.146743
282237   -128.781605
282238   -112.158097
282239   -112.245129
Name: RP, Length: 282240, dtype: float64

In [3]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
min_max_scaler.fit(DATA)
joblib.dump(min_max_scaler, "MLP.scaler.gz") 

TRAINING_DATA = min_max_scaler.transform(TRAINING_DATA)
TRAINING_DATA = pd.DataFrame(TRAINING_DATA)
VALIDATION_DATA = min_max_scaler.transform(VALIDATION_DATA)
VALIDATION_DATA = pd.DataFrame(VALIDATION_DATA)
TEST_DATA = min_max_scaler.transform(TEST_DATA)
TEST_DATA = pd.DataFrame(TEST_DATA)

In [4]:
model = keras.models.Sequential([
    keras.layers.Dense(320,activation="tanh", input_dim=4),
    keras.layers.Dense(1, activation="linear")
])

optimizer = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss="mean_squared_error", optimizer=optimizer)

epochs=20000
batch=100


model_name = f"{model.layers[0].input_shape[1]}"
for layer in model.layers:
    model_name+=f"-{layer.output_shape[1]}"
model_name += f"_e:{epochs}_b:{batch}"
print(model_name)


history = model.fit(TRAINING_DATA, TRAINING_VAL, epochs=epochs, batch_size=batch, validation_data=(VALIDATION_DATA, VALIDATION_VAL))
model.save(f"MODELS/{model_name}.h5")

from math import log10

scores = model.evaluate(TEST_DATA, TEST_VAL, batch_size=20)
plt.plot([10*log10(loss) for loss in history.history['loss'][4:]])
plt.plot([10*log10(loss) for loss in history.history['val_loss'][4:]])
plt.title(f'model loss {scores}')
plt.ylabel('loss dB')
plt.xlabel('epoch')
plt.legend(['TRAIN', 'VALID'], loc='upper left')
plt.savefig(f"TRAINING_PROCESS/{model_name}.png")
plt.show()


4-320-1_e:20000_b:100
Epoch 1/20000


/Users/shsf/.local/share/virtualenvs/code-gcDAJfEO/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1976/1976 [==============================] - 2s 1ms/step - loss: 10835.5107 - val_loss: 5952.3184
Epoch 2/20000
1976/1976 [==============================] - 2s 1ms/step - loss: 2631.2361 - val_loss: 665.6663
Epoch 3/20000
1976/1976 [==============================] - 2s 850us/step - loss: 321.5901 - val_loss: 200.8611
Epoch 4/20000
1976/1976 [==============================] - 2s 887us/step - loss: 179.2006 - val_loss: 144.1973
Epoch 5/20000
1976/1976 [==============================] - 2s 828us/step - loss: 120.2096 - val_loss: 92.8506
Epoch 6/20000
1976/1976 [==============================] - 2s 878us/step - loss: 78.7560 - val_loss: 67.0459
Epoch 7/20000
1976/1976 [==============================] - 2s 883us/step - loss: 62.9950 - val_loss: 60.6993
Epoch 8/20000
1976/1976 [==============================] - 2s 851us/step - loss: 59.9219 - val_loss: 59.7013
Epoch 9/20000
1976/1976 [==============================] - 2s 863us/step - loss: 59.1835 - val_loss: 59.1042
Epoch 10/20000
1976/1976

In [16]:
TEST_DATA = TEST_DATA.iloc[:30,:]
PREDICTIONS = pd.DataFrame(model.predict(TEST_DATA))
PREDICTIONS.columns=["PREDICITONS"]

TEST_VAL = TEST_VAL.iloc[:30,:]
TEST_VAL.columns=["TEST_VAL"]
TEST_VAL = TEST_VAL.reset_index(drop=True)

result=pd.concat([PREDICTIONS, TEST_VAL], axis=1)
result["ERROR"] = result["PREDICITONS"] - result["TEST_VAL"]
result.to_csv("MLP.csv")
